## Importing the files

Load the *libraries.csv files and *.vuln.csv

In [9]:
import pandas as pd
import glob

# Load all libraries files
library_files = glob.glob('*.libraries.csv')
df_libraries_list = []
for file in library_files:
    df_temp = pd.read_csv(file)
    df_temp['source_file'] = file
    df_libraries_list.append(df_temp)

libraries_df = pd.concat(df_libraries_list, ignore_index=True)

# Load all vulnerability files
vuln_files = glob.glob('*.vuln.csv')
df_vulns_list = []
for file in vuln_files:
    df_temp = pd.read_csv(file)
    df_temp['source_file'] = file
    df_vulns_list.append(df_temp)

vulnerabilities_df = pd.concat(df_vulns_list, ignore_index=True)

print(f"{len(libraries_df)} library records loaded from {len(library_files)} files")
print(f"{len(vulnerabilities_df)} vulnerability loaded records from {len(vuln_files)} files")

7338 library records loaded from 10 files
176 vulnerability loaded records from 10 files


## List of libraries used by OSS Projects

In [ ]:
project_counts = libraries_df['Project Name'].value_counts()
print(project_counts)

Project Name
socket.io      1373
jest           1227
mongoose       1103
axios           844
dotenv          773
chalk           750
lodash          699
bcrypt          293
tailwindcss     237
passport         39
Name: count, dtype: int64


## Top Libraries most used

In [ ]:
library_counts = libraries_df['Library Name'].value_counts()
print(library_counts.head(15))

Library Name
semver             60
ansi-styles        58
strip-ansi         46
chalk              46
minimatch          45
debug              44
type-fest          42
supports-color     42
brace-expansion    41
string-width       41
ansi-regex         39
glob               37
lru-cache          35
p-limit            35
minipass           35
Name: count, dtype: int64


## Most used ibraries by OSS projects

In [ ]:
library_analysis = libraries_df.groupby('Library Name').agg({
    'Project Name': ['count', 'nunique']
}).reset_index()

# Flatten column names
library_analysis.columns = ['Library Name', 'Total_Usage', 'Used_by_Projects']

# Sort by total usage
library_analysis = library_analysis.sort_values('Total_Usage', ascending=False)

print(library_analysis.head(10))

         Library Name  Total_Usage  Used_by_Projects
1878           semver           60                 8
33        ansi-styles           58                 9
278             chalk           46                 8
1992       strip-ansi           46                 9
1284        minimatch           45                10
443             debug           44                10
2006   supports-color           42                 9
2116        type-fest           42                 8
220   brace-expansion           41                 9
1984     string-width           41                 9


## Grap: Most used libraries

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.io import push_notebook

# Enable Bokeh in Colab
output_notebook()

# Your data
library_analysis = libraries_df.groupby('Library Name').agg({
    'Project Name': ['count', 'nunique']
}).reset_index()

library_analysis.columns = ['Library Name', 'Total_Usage', 'Unique_Libraries']
library_analysis = library_analysis.sort_values('Total_Usage', ascending=False)

# Get top 20 libraries
top_20 = library_analysis.head(20)

# Create bokeh chart
source = ColumnDataSource(top_20)

p = figure(x_range=list(top_20['Library Name']),
           title="Top 20 Most Used Libraries in key Node.js Projects",
           width=800, height=400)

p.vbar(x='Library Name', top='Total_Usage', width=0.8, source=source)
p.xaxis.major_label_orientation = 45

show(p)

## Libraries with multiple versions

In [ ]:
library_version_analysis = libraries_df.groupby('Library Name')['Library Version'].agg(['nunique', 'unique']).reset_index()
library_version_analysis.columns = ['Library', 'Version_Count', 'All_Versions']

# Libraries with multiple versions across projects
multi_versions = library_version_analysis[library_version_analysis['Version_Count'] > 1].sort_values('Version_Count', ascending=False)
print(multi_versions.head(10))

         Library  Version_Count  \
400         core             24   
2127       types             17   
2116   type-fest             15   
339    commander             12   
443        debug             11   
1936  source-map             10   
2274       yargs             10   
768      globals             10   
1878      semver              9   
1341        node              9   

                                           All_Versions  
400   [7.28.3, 0.2.9, 29.7.0, 7.26.7, 7.24.7, 2.1.6,...  
2127  [7.28.4, 8.43.0, 6.21.0, 29.6.3, 7.26.7, 7.24....  
2116  [0.18.1, 0.8.1, 0.20.2, 0.21.3, 0.6.0, 0.3.1, ...  
339   [9.4.0, 2.15.1, 2.20.3, 8.3.0, 4.1.1, 2.3.0, 0...  
443   [2.6.9, 4.4.1, 4.1.12, 3.2.7, 4.4.0, 2.2.0, 3....  
1936  [0.6.1, 0.3.11, 29.6.3, 0.8.0-beta.0, 0.7.4, 0...  
2274  [15.4.1, 17.7.2, 16.2.0, 17.0.33, 17.7.1, 17.0...  
768   [13.24.0, 14.0.0, 29.7.0, 11.12.0, 8.39.1, 13....  
1878  [6.3.1, 7.7.2, 5.7.2, 7.7.1, 7.6.2, 7.6.3, 7.7...  
1341  [24.3.1, 16.18.126, 22.13.1,

## Libraries with more vulnerabilities

In [15]:
# Merge the dataframes on library name and version
merged_df = libraries_df.merge(
    vulnerabilities_df,
    left_on=['Library Name', 'Library Version'],
    right_on=['name', 'version'],
    how='inner'
)

# Count CVEs per library (grouping by library name only)
cve_count = merged_df.groupby('Library Name').agg({
    'cve': 'count'
}).reset_index()

# Rename columns for clarity
cve_count.columns = ['Library Name', 'CVE Count']

# Step 3: Get top 10 unique libraries with most CVEs
top_10_libraries = cve_count.sort_values('CVE Count', ascending=False).head(10)

print(top_10_libraries.to_string(index=False))

   Library Name  CVE Count
brace-expansion         93
         lodash         61
       electron         20
       minimist         18
         tar-fs          9
            tmp          9
            tar          7
        request          5
     handlebars          5
      minimatch          4


## Libraries organized by Severity

In [8]:
# Assuming you already have libraries_df and vulnerabilities_df loaded

# Merge the dataframes on library name and version
merged_df = libraries_df.merge(
    vulnerabilities_df,
    left_on=['Library Name', 'Library Version'],
    right_on=['name', 'version'],
    how='inner'
)

# Create severity pivot table with all possible severity levels
severity_pivot = merged_df.groupby(['Library Name', 'severity']).size().unstack(fill_value=0)

# Ensure all severity columns exist (add missing ones with 0 values)
severity_columns = ['CRITICAL', 'HIGH', 'LOW', 'MODERATE', 'UNKNOWN']
for col in severity_columns:
    if col not in severity_pivot.columns:
        severity_pivot[col] = 0

# Select and reorder columns
severity_pivot = severity_pivot[severity_columns]

# Calculate total column (sum of all severity counts)
severity_pivot['total'] = severity_pivot.sum(axis=1)

# Sort by total and get top 10
top_10_by_severity = severity_pivot.sort_values('total', ascending=False).head(10)

print(top_10_by_severity.to_string())

severity         CRITICAL  HIGH  LOW  MODERATE  UNKNOWN  total
Library Name                                                  
brace-expansion         0     0   93         0        0     93
lodash                  3    35    0        23        0     61
electron                1     6    3        10        0     20
minimist                9     0    0         9        0     18
handlebars              3     5    0         1        0      9
tmp                     0     0    9         0        0      9
tar-fs                  0     9    0         0        0      9
tar                     0     6    0         1        0      7
request                 0     0    0         5        0      5
minimatch               0     4    0         0        0      4


## Most used libraries by OSS projects with vulnerabilities



In [2]:
# Library analysis with vulnerability counts
library_analysis = libraries_df.groupby('Library Name').agg({
    'Project Name': ['count', 'nunique']
}).reset_index()

# Flatten column names
library_analysis.columns = ['Library Name', 'Total_Usage', 'Unique_Projects']

# Add vulnerability counts by merging with vulnerabilities_df
vuln_counts = vulnerabilities_df.groupby('name').size().reset_index(name='Vuln_Count')

# Merge library analysis with vulnerability counts
library_analysis = library_analysis.merge(
    vuln_counts,
    left_on='Library Name',
    right_on='name',
    how='left'
).drop('name', axis=1)

# Fill NaN values with 0 (libraries with no vulnerabilities)
library_analysis['Vuln_Count'] = library_analysis['Vuln_Count'].fillna(0).astype(int)

# Sort by total usage
library_analysis = library_analysis.sort_values('Unique_Projects', ascending=False)

print(library_analysis.head(10))

         Library Name  Total_Usage  Unique_Projects  Vuln_Count
443             debug           44               10           2
1184        lru-cache           35               10           0
760              glob           37               10           0
1284        minimatch           45               10           3
1317               ms           34               10           1
2257        wrap-ansi           30                9           0
1895  shebang-command            9                9           0
416       cross-spawn            9                9           2
988             isexe           18                9           0
1896    shebang-regex            9                9           0


## OSS Projects with more vulnerabilities by severity

In [ ]:
# Projects with vulnerabilities by severity
affected_projects = libraries_df.merge(
    vulnerabilities_df,
    left_on=['Library Name', 'Library Version'],
    right_on=['name', 'version'],
    how='inner'
)

# Count vulnerabilities by project and severity
project_vulns = affected_projects.groupby(['Project Name', 'severity'])['cve'].nunique().unstack(fill_value=0)

# Add total column
project_vulns['Total'] = project_vulns.sum(axis=1)

# Sort by total vulnerabilities
project_vulns = project_vulns.sort_values('Total', ascending=False)

print(project_vulns)

severity      CRITICAL  HIGH  LOW  MODERATE  UNKNOWN  Total
Project Name                                               
lodash              13    34    3        19        0     69
socket.io            1     9    8         4        0     22
chalk                1     6    3        11        0     21
passport             2     3    1         2        0      8
mongoose             0     2    1         2        0      5
axios                0     0    2         0        0      2
bcrypt               0     1    1         0        0      2
dotenv               0     0    1         1        0      2
tailwindcss          0     0    0         0        0      0


## Graph: Projects with vulnerabilities by severity

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource

# Enable Bokeh in Colab
output_notebook()

# Your data
affected_projects = libraries_df.merge(
    vulnerabilities_df,
    left_on=['Library Name', 'Library Version'],
    right_on=['name', 'version'],
    how='inner'
)

project_vulns = affected_projects.groupby(['Project Name', 'severity'])['cve'].nunique().unstack(fill_value=0)
project_vulns['Total'] = project_vulns.sum(axis=1)
project_vulns = project_vulns.sort_values('Total', ascending=False)

# Get top 10 projects
top_projects = project_vulns.head(10).reset_index()

# Create figure
p = figure(y_range=list(top_projects['Project Name']),
           title="Projects with Vulnerabilities by Severity",
           width=800, height=500)

# Get available severity columns
severity_cols = [col for col in top_projects.columns if col not in ['Project Name', 'Total']]
colors = ['red', 'orange', 'yellow', 'green', 'blue']

# Simple horizontal bars for each severity
for i, severity in enumerate(severity_cols):
    if severity in top_projects.columns:
        source = ColumnDataSource(top_projects)
        p.hbar(y='Project Name', right=severity, height=0.8,
               source=source, color=colors[i % len(colors)],
               legend_label=severity, alpha=0.7)

p.legend.location = "top_right"
p.xaxis.axis_label = "Number of Vulnerabilities"

show(p)

## Top licenses used by libraries

In [ ]:
library_license_analysis = libraries_df.groupby('Library License').agg({
    'Library Name': ['count', 'nunique']
}).reset_index()

# Flatten column names
library_license_analysis.columns = ['Library Name', 'Total_Usage', 'Unique_Libraries']

# Sort by total usage
library_license_analysis = library_license_analysis.sort_values('Total_Usage', ascending=False)

print(library_license_analysis.head(17))

     Library Name  Total_Usage  Unique_Libraries
14            MIT         5735              1903
13            ISC          807               168
3      Apache-2.0          209                79
6    BSD-3-Clause          164                43
5    BSD-2-Clause          135                34
8   BlueOak-1.0.0           61                37
4             BSD           18                16
18     Python-2.0           10                 1
11        CC0-1.0            7                 2
10      CC-BY-4.0            7                 2
0            0BSD            6                 1
2      Apache 2.0            6                 4
9       CC-BY-3.0            6                 1
21          WTFPL            5                 5
20      Unlicense            4                 4
7        BSD-like            4                 3
15        MIT/X11            3                 3


## Top Authors of libraries

In [ ]:
library_author_analysis = libraries_df.groupby('Library Author').agg({
    'Library Name': ['count', 'nunique']
}).reset_index()

# Flatten column names
library_author_analysis.columns = ['Library Author Name', 'Total_Usage', 'Unique_Libraries']

# Sort by total usage
library_author_analysis = library_author_analysis.sort_values('Total_Usage', ascending=False)

print(library_author_analysis.head(10))

    Library Author Name  Total_Usage  Unique_Libraries
389       Sindre Sorhus         1249               223
174  Isaac Z. Schlueter          388                81
411      The Babel Team          345               112
222      Jordan Harband          231                99
151         GitHub Inc.          183                56
216      Jon Schlinkert          139                55
48              Ben Coe          111                14
425        Titus Wormer          102                62
405      TJ Holowaychuk           88                21
293      Mathias Bynens           87                14


 ## Libraries with missing author info

In [ ]:
library_unknown_authors = libraries_df[libraries_df['Library Author'].isna()]
unknown_summary = library_unknown_authors.groupby('Library Name').agg({
    'Project Name': 'nunique',
    'Library Version': 'first'
}).rename(columns={'Project Name': 'Used_In_Projects', 'Library Version': 'Version'})
print(unknown_summary.sort_values('Used_In_Projects', ascending=False).head(20))

                    Used_In_Projects Version
Library Name                                
ms                                10   2.0.0
inherits                           9   2.0.4
argparse                           8  1.0.10
chalk                              8   4.1.2
source-map-support                 8  0.5.21
yargs                              8  15.4.1
error-ex                           8   1.3.2
graceful-fs                        8  4.2.11
fsevents                           8   2.3.3
acorn                              8  8.15.0
string_decoder                     7   1.1.1
buffer-from                        7   1.1.2
readable-stream                    7   3.6.2
neo-async                          7   2.6.2
node                               7  24.3.1
estraverse                         7   4.3.0
esutils                            7   2.0.3
merge2                             6   1.4.1
mime-types                         6  2.1.35
commons                            6   3.0.1


## Libraries with multiple versions and no author info

In [ ]:
# Simple risky libraries summary
risky_analysis = libraries_df.groupby('Library Name').agg({
    'Library Version': 'nunique',
    'Library Author': lambda x: x.isna().sum()
}).rename(columns={
    'Library Version': 'Version_Count',
    'Library Author': 'No_Author_Count'
})

# Add vulnerability counts
vuln_counts = vulnerabilities_df.groupby('name').size().reset_index(name='Vuln_Count')
risky_analysis = risky_analysis.merge(vuln_counts, left_index=True, right_on='name', how='left')
risky_analysis['Vuln_Count'] = risky_analysis['Vuln_Count'].fillna(0).astype(int)
risky_analysis = risky_analysis.set_index('name')

# Sort by risk (multiple criteria)
risky_analysis = risky_analysis.sort_values(['Version_Count', 'Vuln_Count', 'No_Author_Count'], ascending=False)

print("Library Name | Versions | No Author | Vulnerabilities")
print(risky_analysis.head(20))

Library Name | Versions | No Author | Vulnerabilities
                 Version_Count  No_Author_Count  Vuln_Count
name                                                       
core                        24                4           0
types                       17               12           0
type-fest                   15                0           0
commander                   12                0           0
debug                       11                5           2
yargs                       10               21           0
globals                     10                4           0
source-map                  10                3           0
semver                       9                4           1
chalk                        9               45           0
node                         9               22           0
readable-stream              9               19           0
glob                         9                1           0
lru-cache                    9                

## Generate a dataset with all libraries and version

In [ ]:
# Export all libraries to CSV
all_libraries = libraries_df[['Library Name', 'Library Version']].drop_duplicates()

# Save to CSV
all_libraries.to_csv('all_libraries.csv', index=False)

print(f"Exported {len(all_libraries)} unique library-version combinations to 'all_libraries.csv'")
print("\nFirst 10 rows:")
print(all_libraries.head(10))

Exported 3870 unique library-version combinations to 'all_libraries.csv'

First 10 rows:
         Library Name Library Version
0                core          7.28.3
1          preset-env          7.28.3
2  mongodb-downloader           0.4.2
3       eslint-plugin          8.43.0
4              parser          8.43.0
5       acquit-ignore           0.2.1
6      acquit-require           0.1.1
7              acquit           1.4.0
8                 ajv          8.17.1
9   assert-browserify           2.0.0


## Get the Current version of libraries

In [ ]:
import pandas as pd
import requests
from packaging import version
import re

# Load your CSV
df = pd.read_csv('all_libraries.csv')

def clean_version(ver_string):
    """Clean version string to make it parseable"""
    if not ver_string or pd.isna(ver_string):
        return "0.0.0"

    ver_string = str(ver_string).strip()

    # Remove problematic suffixes
    ver_string = re.sub(r'-security.*', '', ver_string)
    ver_string = re.sub(r'-alpha.*', '', ver_string)
    ver_string = re.sub(r'-beta.*', '', ver_string)
    ver_string = re.sub(r'-rc.*', '', ver_string)
    ver_string = re.sub(r'-SNAPSHOT.*', '', ver_string)

    # Keep only digits and dots
    ver_string = re.sub(r'[^\d\.]', '', ver_string)

    # Ensure x.y.z format
    parts = ver_string.split('.')
    while len(parts) < 3:
        parts.append('0')

    return '.'.join(parts[:3])

def get_latest_npm_version(package_name):
    url = f'https://registry.npmjs.org/{package_name}'
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            return data.get("dist-tags", {}).get("latest")
    except:
        pass
    return None

def safe_version_compare(current_version, latest_version):
    if latest_version is None:
        return "Not Found"

    try:
        current_clean = clean_version(current_version)
        latest_clean = clean_version(latest_version)

        if version.parse(current_clean) < version.parse(latest_clean):
            return f"Update available ({latest_version})"
        elif version.parse(current_clean) == version.parse(latest_clean):
            return "Up-to-date"
        else:
            return f"Current version is newer than npm ({latest_version})"
    except:
        return "Version comparison failed"

# Check each library
results = []
for i, row in df.iterrows():
    package = row['Library Name']
    current_version = row['Library Version']

    print(f"Checking {i+1}/{len(df)}: {package}")

    latest_version = get_latest_npm_version(package)
    status = safe_version_compare(current_version, latest_version)

    results.append({
        "Library": package,
        "Current Version": current_version,
        "Latest Version": latest_version,
        "Status": status
    })

# Convert to DataFrame and display
results_df = pd.DataFrame(results)
print("\nResults:")
print(results_df)

# Save to CSV
results_df.to_csv("npm_version_check_results.csv", index=False)
print(f"\nSaved results to npm_version_check_results.csv")

Checking 1/3870: core
Checking 2/3870: preset-env
Checking 3/3870: mongodb-downloader
Checking 4/3870: eslint-plugin
Checking 5/3870: parser
Checking 6/3870: acquit-ignore
Checking 7/3870: acquit-require
Checking 8/3870: acquit
Checking 9/3870: ajv
Checking 10/3870: assert-browserify
Checking 11/3870: babel-loader
Checking 12/3870: broken-link-checker
Checking 13/3870: bson
Checking 14/3870: buffer
Checking 15/3870: cheerio
Checking 16/3870: parse5
Checking 17/3870: entities
Checking 18/3870: crypto-browserify
Checking 19/3870: dox
Checking 20/3870: commander
Checking 21/3870: eslint-plugin-markdown
Checking 22/3870: eslint-plugin-mocha-no-only
Checking 23/3870: eslint
Checking 24/3870: ajv
Checking 25/3870: chalk
Checking 26/3870: escape-string-regexp
Checking 27/3870: ignore
Checking 28/3870: minimatch
Checking 29/3870: strip-ansi
Checking 30/3870: json-schema-traverse
Checking 31/3870: ansi-styles
Checking 32/3870: supports-color
Checking 33/3870: brace-expansion
Checking 34/3870: a

## Get the most oudated libraries

In [7]:
import pandas as pd
from packaging import version
import re

# Load the CSV file
df = pd.read_csv('npm_version_check_results.csv')

def clean_version_for_comparison(ver_string):
    """Clean version string to make it comparable"""
    if not ver_string or pd.isna(ver_string):
        return "0.0.0"

    ver_string = str(ver_string).strip()

    # Remove problematic suffixes
    ver_string = re.sub(r'-security.*', '', ver_string)
    ver_string = re.sub(r'-alpha.*', '', ver_string)
    ver_string = re.sub(r'-beta.*', '', ver_string)
    ver_string = re.sub(r'-rc.*', '', ver_string)
    ver_string = re.sub(r'-SNAPSHOT.*', '', ver_string)

    # Keep only digits and dots
    ver_string = re.sub(r'[^\d\.]', '', ver_string)

    # Ensure x.y.z format
    parts = ver_string.split('.')
    while len(parts) < 3:
        parts.append('0')

    return '.'.join(parts[:3])

def calculate_version_gap(current_ver, latest_ver):
    """Calculate how far behind the current version is"""
    try:
        current_clean = clean_version_for_comparison(current_ver)
        latest_clean = clean_version_for_comparison(latest_ver)

        current_parsed = version.parse(current_clean)
        latest_parsed = version.parse(latest_clean)

        # Calculate major.minor difference as a rough metric
        current_parts = [int(x) for x in current_clean.split('.')]
        latest_parts = [int(x) for x in latest_clean.split('.')]

        major_diff = latest_parts[0] - current_parts[0]
        minor_diff = latest_parts[1] - current_parts[1]
        patch_diff = latest_parts[2] - current_parts[2]

        # Weight major changes more heavily
        gap_score = major_diff * 100 + minor_diff * 10 + patch_diff
        return gap_score
    except:
        return 0

# Filter for libraries that have updates available
outdated_df = df[df['Status'].str.contains('Update available', na=False)].copy()

# Calculate version gap for each outdated library
outdated_df['Version_Gap'] = outdated_df.apply(
    lambda row: calculate_version_gap(row['Current Version'], row['Latest Version']),
    axis=1
)

# Sort by version gap (biggest gaps first) and take top 10
top_outdated = outdated_df.nlargest(10, 'Version_Gap')

print(f"{'Library Name':<30} {'Current':<12} {'Latest':<12}")

for _, row in top_outdated.iterrows():
    print(f"{row['Library']:<30} {row['Current Version']:<12} {row['Latest Version']:<12}")

print(f"\nTotal outdated libraries: {len(outdated_df)} out of {len(df)}")

Library Name                   Current      Latest      
devtools-protocol              0.0.1120988  0.0.1512837 
devtools-protocol              0.0.1147663  0.0.1512837 
devtools-protocol              0.0.1302984  0.0.1512837 
api-extractor                  7.52.8       99.99.99    
electron                       0.4.1        38.0.0      
jest                           2.1.1        30.1.3      
expect                         2.1.9        30.1.2      
pretty-format                  2.1.9        30.0.5      
node                           0.16.6       20.19.5     
react                          2.0.0        19.1.1      

Total outdated libraries: 1799 out of 3870


## The top most updated libraries

In [9]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('npm_version_check_results.csv')

# Show up-to-date libraries (limit to top 10)
updated_df = df[df['Status'].isin(['Up-to-date', 'Current version is newer than npm'])]

print(f"{'Library Name':<30} {'Current':<12} {'Latest':<12}")

# Show only first 10 up-to-date libraries
for _, row in updated_df.head(10).iterrows():
    print(f"{row['Library']:<30} {row['Current Version']:<12} {row['Latest Version']:<12}")

print(f"\nTotal up-to-date libraries: {len(updated_df)} out of {len(df)}")

Library Name                   Current      Latest      
acquit-ignore                  0.2.1        0.2.1       
acquit-require                 0.1.1        0.1.1       
acquit                         1.4.0        1.4.0       
ajv                            8.17.1       8.17.1      
assert-browserify              2.0.0        2.0.0       
broken-link-checker            0.7.8        0.7.8       
bson                           6.10.4       6.10.4      
cheerio                        1.1.2        1.1.2       
crypto-browserify              3.12.1       3.12.1      
dox                            1.0.0        1.0.0       

Total up-to-date libraries: 1298 out of 3870


## Graph: Risk Assessment by OSS Project

In [18]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool, LabelSet
import pandas as pd

# Enable Bokeh in Colab
output_notebook()

# Load version results
version_results = pd.read_csv('npm_version_check_results.csv')
outdated_libraries = set(version_results[version_results['Status'].str.contains('Update available', na=False)]['Library'])

# Define severity weights for risk calculation
severity_weights = {
    'CRITICAL': 20,
    'HIGH': 10,
    'MODERATE': 5,
    'LOW': 2
}

# Calculate risk for each project
project_data = []

for project in libraries_df['Project Name'].unique():
    project_libs = libraries_df[libraries_df['Project Name'] == project]

    # Merge with vulnerabilities
    project_vulns = project_libs.merge(
        vulnerabilities_df,
        left_on=['Library Name', 'Library Version'],
        right_on=['name', 'version'],
        how='inner'
    )

    # Count vulnerabilities by severity
    severity_counts = project_vulns['severity'].value_counts()
    critical_count = severity_counts.get('CRITICAL', 0)
    high_count = severity_counts.get('HIGH', 0)
    moderate_count = severity_counts.get('MODERATE', 0)
    low_count = severity_counts.get('LOW', 0)

    # Calculate weighted vulnerability score
    vuln_score = (critical_count * severity_weights['CRITICAL'] +
                  high_count * severity_weights['HIGH'] +
                  moderate_count * severity_weights['MODERATE'] +
                  low_count * severity_weights['LOW'])

    # Count outdated libraries
    outdated_count = project_libs['Library Name'].isin(outdated_libraries).sum()

    # Count missing authors
    author_missing = project_libs['Library Author'].isna().sum()

    # Calculate total risk score
    risk_score = vuln_score + (outdated_count * 3) + (author_missing * 1)
    library_count = len(project_libs)
    total_vulns = len(project_vulns)

    project_data.append({
        'Project': project,
        'Risk_Score': risk_score,
        'Library_Count': library_count,
        'Total_Vulns': total_vulns,
        'Critical': critical_count,
        'High': high_count,
        'Moderate': moderate_count,
        'Low': low_count,
        'Outdated': outdated_count,
        'No_Author': author_missing,
        'Vuln_Score': vuln_score
    })

# Create DataFrame
df = pd.DataFrame(project_data)

# Calculate medians for quadrants
risk_median = df['Risk_Score'].median()
lib_median = df['Library_Count'].median()

# Assign colors based on quadrants
def get_color(risk, libs):
    if risk <= risk_median and libs <= lib_median:
        return 'green'    # Low risk, few libs
    elif risk <= risk_median and libs > lib_median:
        return 'yellow'   # Low risk, many libs
    elif risk > risk_median and libs <= lib_median:
        return 'orange'   # High risk, few libs
    else:
        return 'red'      # High risk, many libs

df['Color'] = df.apply(lambda row: get_color(row['Risk_Score'], row['Library_Count']), axis=1)

# Create plot
source = ColumnDataSource(df)

p = figure(title="Project Risk Assessment - Enhanced with Severity Weights",
           x_axis_label="Risk Score (Weighted by Severity)",
           y_axis_label="Library Count",
           width=800, height=600)

p.scatter('Risk_Score', 'Library_Count', size=12, alpha=0.7,
          color='Color', source=source)

# Add project names above points
labels = LabelSet(x='Risk_Score', y='Library_Count', text='Project',
                 x_offset=0, y_offset=12, source=source,
                 text_font_size="9pt", text_color="black", text_align="center")
p.add_layout(labels)

# Add quadrant lines
p.line([risk_median, risk_median], [0, df['Library_Count'].max()],
       line_color='gray', line_dash='dashed', alpha=0.5, line_width=2)
p.line([0, df['Risk_Score'].max()], [lib_median, lib_median],
       line_color='gray', line_dash='dashed', alpha=0.5, line_width=2)

# Enhanced hover with severity breakdown
hover = HoverTool(tooltips=[
    ('Project', '@Project'),
    ('Risk Score', '@Risk_Score'),
    ('Libraries', '@Library_Count'),
    ('Total Vulns', '@Total_Vulns'),
    ('Critical', '@Critical'),
    ('High', '@High'),
    ('Moderate', '@Moderate'),
    ('Low', '@Low'),
    ('Outdated', '@Outdated'),
    ('No Author', '@No_Author'),
    ('Vuln Score', '@Vuln_Score')
])
p.add_tools(hover)

show(p)

print(f"\nTop Highest Risk Projects:")
top_risk = df.nlargest(4, 'Risk_Score')[['Project', 'Risk_Score', 'Critical', 'High', 'Outdated']]
print(top_risk.to_string(index=False))


Top Highest Risk Projects:
  Project  Risk_Score  Critical  High  Outdated
socket.io        3373         1    16       943
   lodash        3331        23    84       513
     jest        2387         0     0       703
 mongoose        2249         0     2       687


## OSS Projects Risk Compositions

In [40]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
import pandas as pd

# Enable Bokeh in Colab
output_notebook()

# Load version results
version_results = pd.read_csv('npm_version_check_results.csv')
outdated_libraries = set(version_results[version_results['Status'].str.contains('Update available', na=False)]['Library'])

# Define severity weights
severity_weights = {
    'CRITICAL': 20,
    'HIGH': 10,
    'MODERATE': 5,
    'LOW': 2
}

# Calculate risk for each project
project_data = []

for project in libraries_df['Project Name'].unique():
    project_libs = libraries_df[libraries_df['Project Name'] == project]

    # Merge with vulnerabilities
    project_vulns = project_libs.merge(
        vulnerabilities_df,
        left_on=['Library Name', 'Library Version'],
        right_on=['name', 'version'],
        how='inner'
    )

    # Count vulnerabilities by severity
    severity_counts = project_vulns['severity'].value_counts()
    critical_count = severity_counts.get('CRITICAL', 0)
    high_count = severity_counts.get('HIGH', 0)
    moderate_count = severity_counts.get('MODERATE', 0)
    low_count = severity_counts.get('LOW', 0)

    # Calculate weighted vulnerability score
    vuln_score = (critical_count * severity_weights['CRITICAL'] +
                  high_count * severity_weights['HIGH'] +
                  moderate_count * severity_weights['MODERATE'] +
                  low_count * severity_weights['LOW'])

    # Count outdated libraries and missing authors
    outdated_count = project_libs['Library Name'].isin(outdated_libraries).sum()
    author_missing = project_libs['Library Author'].isna().sum()

    # Calculate scores for each component
    critical_score = critical_count * 20
    high_score = high_count * 10
    moderate_score = moderate_count * 5
    low_score = low_count * 2
    outdated_score = outdated_count * 3
    missing_score = author_missing * 1

    # Calculate total risk score
    total_risk_score = critical_score + high_score + moderate_score + low_score + outdated_score + missing_score

    project_data.append({
        'Project': project,
        'Risk_Score': total_risk_score,
        'Critical': critical_count,
        'High': high_count,
        'Moderate': moderate_count,
        'Low': low_count,
        'Outdated': outdated_count,
        'Missing_Authors': author_missing,
        'Critical_Score': critical_score,
        'High_Score': high_score,
        'Moderate_Score': moderate_score,
        'Low_Score': low_score,
        'Outdated_Score': outdated_score,
        'Missing_Score': missing_score
    })

# Create DataFrame and sort by risk score
df = pd.DataFrame(project_data)
df = df.sort_values('Risk_Score', ascending=False)

# Calculate stacking positions
df['critical_bottom'] = 0
df['critical_top'] = df['Critical_Score']

df['high_bottom'] = df['critical_top']
df['high_top'] = df['high_bottom'] + df['High_Score']

df['moderate_bottom'] = df['high_top']
df['moderate_top'] = df['moderate_bottom'] + df['Moderate_Score']

df['low_bottom'] = df['moderate_top']
df['low_top'] = df['low_bottom'] + df['Low_Score']

df['outdated_bottom'] = df['low_top']
df['outdated_top'] = df['outdated_bottom'] + df['Outdated_Score']

df['missing_bottom'] = df['outdated_top']
df['missing_top'] = df['missing_bottom'] + df['Missing_Score']

# Create the stacked bar chart
source = ColumnDataSource(df)

p = figure(x_range=df['Project'].tolist(),
           title="OSS Projects Risk Composition - Stacked by Risk Components",
           x_axis_label="Projects",
           y_axis_label="Risk Score",
           width=1200, height=600)

# Create stacked bars - only add if value > 0 to avoid empty segments
if df['Critical_Score'].sum() > 0:
    p.vbar(x='Project', bottom='critical_bottom', top='critical_top', width=0.8,
           color='#8B0000', source=source, alpha=0.9, legend_label="Critical CVEs")

if df['High_Score'].sum() > 0:
    p.vbar(x='Project', bottom='high_bottom', top='high_top', width=0.8,
           color='#FF0000', source=source, alpha=0.9, legend_label="High CVEs")

if df['Moderate_Score'].sum() > 0:
    p.vbar(x='Project', bottom='moderate_bottom', top='moderate_top', width=0.8,
           color='#FF8C00', source=source, alpha=0.9, legend_label="Moderate CVEs")

if df['Low_Score'].sum() > 0:
    p.vbar(x='Project', bottom='low_bottom', top='low_top', width=0.8,
           color='#FFD700', source=source, alpha=0.9, legend_label="Low CVEs")

if df['Outdated_Score'].sum() > 0:
    p.vbar(x='Project', bottom='outdated_bottom', top='outdated_top', width=0.8,
           color='#4169E1', source=source, alpha=0.9, legend_label="Outdated Libraries")

if df['Missing_Score'].sum() > 0:
    p.vbar(x='Project', bottom='missing_bottom', top='missing_top', width=0.8,
           color='#808080', source=source, alpha=0.9, legend_label="Missing Authors")

# Configure chart
p.xaxis.major_label_orientation = 45
p.legend.location = "top_right"
p.legend.click_policy = "hide"

# Add hover tooltip
hover = HoverTool(tooltips=[
    ('Project', '@Project'),
    ('Total Risk Score', '@Risk_Score'),
    ('', ''),
    ('Critical CVEs', '@Critical (@Critical_Score pts)'),
    ('High CVEs', '@High (@High_Score pts)'),
    ('Moderate CVEs', '@Moderate (@Moderate_Score pts)'),
    ('Low CVEs', '@Low (@Low_Score pts)'),
    ('Outdated Libraries', '@Outdated (@Outdated_Score pts)'),
    ('Missing Authors', '@Missing_Authors (@Missing_Score pts)')
])
p.add_tools(hover)

show(p)
print("\n")
print(df[['Project', 'Risk_Score', 'Critical', 'High', 'Moderate', 'Low', 'Outdated', 'Missing_Authors']].to_string(index=False))



    Project  Risk_Score  Critical  High  Moderate  Low  Outdated  Missing_Authors
  socket.io        3373         1    16         4   40       943              264
     lodash        3331        23    84        56   13       513              186
       jest        2387         0     0         0    0       703              278
   mongoose        2249         0     2         2    3       687              152
     dotenv        2096         0     0         1   41       641               86
      chalk        1929         1     6        11    3       571               75
      axios        1803         0     0         0   11       522              215
     bcrypt         714         0     1         0    5       201               91
tailwindcss         467         0     0         1    0       117              111
   passport         214         3     4         3    1        31                4


## Export a html file

In [38]:
from bokeh.plotting import figure, output_file, save
from bokeh.models import ColumnDataSource, HoverTool, LabelSet
from bokeh.layouts import column
from bokeh.models import Div
import pandas as pd

# Set output to HTML file
output_file("index.html")

# Load version results
version_results = pd.read_csv('npm_version_check_results.csv')
outdated_libraries = set(version_results[version_results['Status'].str.contains('Update available', na=False)]['Library'])

# Define severity weights
severity_weights = {
    'CRITICAL': 20,
    'HIGH': 10,
    'MODERATE': 5,
    'LOW': 2
}

# Calculate risk for each project
project_data = []

for project in libraries_df['Project Name'].unique():
    project_libs = libraries_df[libraries_df['Project Name'] == project]

    # Merge with vulnerabilities
    project_vulns = project_libs.merge(
        vulnerabilities_df,
        left_on=['Library Name', 'Library Version'],
        right_on=['name', 'version'],
        how='inner'
    )

    # Count vulnerabilities by severity
    severity_counts = project_vulns['severity'].value_counts()
    critical_count = severity_counts.get('CRITICAL', 0)
    high_count = severity_counts.get('HIGH', 0)
    moderate_count = severity_counts.get('MODERATE', 0)
    low_count = severity_counts.get('LOW', 0)

    # Calculate weighted vulnerability score
    vuln_score = (critical_count * severity_weights['CRITICAL'] +
                  high_count * severity_weights['HIGH'] +
                  moderate_count * severity_weights['MODERATE'] +
                  low_count * severity_weights['LOW'])

    # Count outdated libraries and missing authors
    outdated_count = project_libs['Library Name'].isin(outdated_libraries).sum()
    author_missing = project_libs['Library Author'].isna().sum()

    # Calculate scores for each component
    critical_score = critical_count * 20
    high_score = high_count * 10
    moderate_score = moderate_count * 5
    low_score = low_count * 2
    outdated_score = outdated_count * 3
    missing_score = author_missing * 1

    # Calculate total risk score
    total_risk_score = critical_score + high_score + moderate_score + low_score + outdated_score + missing_score
    library_count = len(project_libs)
    total_vulns = len(project_vulns)

    project_data.append({
        'Project': project,
        'Risk_Score': total_risk_score,
        'Library_Count': library_count,
        'Total_Vulns': total_vulns,
        'Critical': critical_count,
        'High': high_count,
        'Moderate': moderate_count,
        'Low': low_count,
        'Outdated': outdated_count,
        'Missing_Authors': author_missing,
        'Vuln_Score': vuln_score,
        'Critical_Score': critical_score,
        'High_Score': high_score,
        'Moderate_Score': moderate_score,
        'Low_Score': low_score,
        'Outdated_Score': outdated_score,
        'Missing_Score': missing_score
    })

# Create DataFrame
df = pd.DataFrame(project_data)

# GRAPH 1: RISK ASSESSMENT SCATTER PLOT

# Calculate medians for quadrants
risk_median = df['Risk_Score'].median()
lib_median = df['Library_Count'].median()

# Assign colors based on quadrants
def get_color(risk, libs):
    if risk <= risk_median and libs <= lib_median:
        return 'green'    # Low risk, few libs
    elif risk <= risk_median and libs > lib_median:
        return 'yellow'   # Low risk, many libs
    elif risk > risk_median and libs <= lib_median:
        return 'orange'   # High risk, few libs
    else:
        return 'red'      # High risk, many libs

df['Color'] = df.apply(lambda row: get_color(row['Risk_Score'], row['Library_Count']), axis=1)

# Create first plot - scatter plot
source1 = ColumnDataSource(df)

p1 = figure(title="Project Risk Assessment - Enhanced with Severity Weights",
           x_axis_label="Risk Score (Weighted by Severity)",
           y_axis_label="Library Count",
           width=800, height=600)

p1.scatter('Risk_Score', 'Library_Count', size=12, alpha=0.7,
          color='Color', source=source1)

# Add project names above points
labels = LabelSet(x='Risk_Score', y='Library_Count', text='Project',
                 x_offset=0, y_offset=12, source=source1,
                 text_font_size="9pt", text_color="black", text_align="center")
p1.add_layout(labels)

# Add quadrant lines
p1.line([risk_median, risk_median], [0, df['Library_Count'].max()],
       line_color='gray', line_dash='dashed', alpha=0.5, line_width=2)
p1.line([0, df['Risk_Score'].max()], [lib_median, lib_median],
       line_color='gray', line_dash='dashed', alpha=0.5, line_width=2)

# Enhanced hover with severity breakdown
hover1 = HoverTool(tooltips=[
    ('Project', '@Project'),
    ('Risk Score', '@Risk_Score'),
    ('Libraries', '@Library_Count'),
    ('Total Vulns', '@Total_Vulns'),
    ('Critical', '@Critical'),
    ('High', '@High'),
    ('Moderate', '@Moderate'),
    ('Low', '@Low'),
    ('Outdated', '@Outdated'),
    ('Missing Authors', '@Missing_Authors'),
    ('Vuln Score', '@Vuln_Score')
])
p1.add_tools(hover1)

# GRAPH 2: STACKED BAR CHART

# Sort by risk score for bar chart
df_sorted = df.sort_values('Risk_Score', ascending=False)

# Calculate stacking positions
df_sorted['critical_bottom'] = 0
df_sorted['critical_top'] = df_sorted['Critical_Score']

df_sorted['high_bottom'] = df_sorted['critical_top']
df_sorted['high_top'] = df_sorted['high_bottom'] + df_sorted['High_Score']

df_sorted['moderate_bottom'] = df_sorted['high_top']
df_sorted['moderate_top'] = df_sorted['moderate_bottom'] + df_sorted['Moderate_Score']

df_sorted['low_bottom'] = df_sorted['moderate_top']
df_sorted['low_top'] = df_sorted['low_bottom'] + df_sorted['Low_Score']

df_sorted['outdated_bottom'] = df_sorted['low_top']
df_sorted['outdated_top'] = df_sorted['outdated_bottom'] + df_sorted['Outdated_Score']

df_sorted['missing_bottom'] = df_sorted['outdated_top']
df_sorted['missing_top'] = df_sorted['missing_bottom'] + df_sorted['Missing_Score']

# Create second plot - stacked bar chart
source2 = ColumnDataSource(df_sorted)

p2 = figure(x_range=df_sorted['Project'].tolist(),
           title="OSS Projects Risk Composition - Stacked by Risk Components",
           x_axis_label="Projects",
           y_axis_label="Risk Score",
           width=1200, height=600)

# Create stacked bars - only add if value > 0 to avoid empty segments
if df_sorted['Critical_Score'].sum() > 0:
    p2.vbar(x='Project', bottom='critical_bottom', top='critical_top', width=0.8,
           color='#8B0000', source=source2, alpha=0.9, legend_label="Critical CVEs")

if df_sorted['High_Score'].sum() > 0:
    p2.vbar(x='Project', bottom='high_bottom', top='high_top', width=0.8,
           color='#FF0000', source=source2, alpha=0.9, legend_label="High CVEs")

if df_sorted['Moderate_Score'].sum() > 0:
    p2.vbar(x='Project', bottom='moderate_bottom', top='moderate_top', width=0.8,
           color='#FF8C00', source=source2, alpha=0.9, legend_label="Moderate CVEs")

if df_sorted['Low_Score'].sum() > 0:
    p2.vbar(x='Project', bottom='low_bottom', top='low_top', width=0.8,
           color='#FFD700', source=source2, alpha=0.9, legend_label="Low CVEs")

if df_sorted['Outdated_Score'].sum() > 0:
    p2.vbar(x='Project', bottom='outdated_bottom', top='outdated_top', width=0.8,
           color='#4169E1', source=source2, alpha=0.9, legend_label="Outdated Libraries")

if df_sorted['Missing_Score'].sum() > 0:
    p2.vbar(x='Project', bottom='missing_bottom', top='missing_top', width=0.8,
           color='#808080', source=source2, alpha=0.9, legend_label="Missing Authors")

# Configure second chart
p2.xaxis.major_label_orientation = 45
p2.legend.location = "top_right"
p2.legend.click_policy = "hide"

# Add hover tooltip
hover2 = HoverTool(tooltips=[
    ('Project', '@Project'),
    ('Total Risk Score', '@Risk_Score'),
    ('', ''),
    ('Critical CVEs', '@Critical (@Critical_Score pts)'),
    ('High CVEs', '@High (@High_Score pts)'),
    ('Moderate CVEs', '@Moderate (@Moderate_Score pts)'),
    ('Low CVEs', '@Low (@Low_Score pts)'),
    ('Outdated Libraries', '@Outdated (@Outdated_Score pts)'),
    ('Missing Authors', '@Missing_Authors (@Missing_Score pts)')
])
p2.add_tools(hover2)

# CREATE LAYOUT AND SAVE

# Add title and description
title_div = Div(text="""
<h1 style="text-align: center; color: #2F4F4F;">OSS Projects Security Risk Assessment Dashboard</h1>
<p style="text-align: center; font-size: 14px; color: #666;">
Analysis of Open Source Software projects showing risk scores based on CVE severity, outdated libraries, and missing authors.
</p>
<hr>
""", width=1200, height=80)

description1 = Div(text="""
<h2 style="color: #2F4F4F;">Graph 1: Risk vs Library Count Quadrant Analysis</h2>
""", width=800, height=60)

description2 = Div(text="""
<h2 style="color: #2F4F4F; margin-top: 30px; margin-bottom: 10px;">Graph 2: OSS Projects by Risk Composition</h2>
""", width=1000, height=60)

# Add footer
footer_div = Div(text="""
<div style="text-align: center; width: 100%; margin: 50px auto 20px auto;">
<p style="color: #7a7a7a; font-size: 12px;">danbreu.com</p>
</div>
""", width=1200, height=50)

# Combine all elements in a layout with center alignment
layout = column(title_div, description1, p1, description2, p2, footer_div,
                sizing_mode="fixed", margin=(0, 100, 0, 100))  # Center the entire layout

# Save to HTML file
save(layout)

print("Successfully exported both graphs to 'index.html'")

Successfully exported both graphs to 'index.html'
